# Level‑1 Cross‑Architectural Internal State Experiment

This notebook demonstrates **architecture‑independent Level‑1 evidence**:

- Rule‑based controller
- Tiny neural network (fixed random weights)
- Stochastic probabilistic policy

All agents are evaluated for **trajectory divergence conditioned on internal state**.

Conscious Machines Level-1 Research Repository:
[https://github.com/conscious-machines/level-1-research](https://github.com/conscious-machines/level-1-research)

Conscious Machines Research Website:
[https://research.conscious-machines.org](https://research.conscious-machines.org)


In [ ]:
import numpy as np
from scipy.stats import chi2_contingency
np.random.seed(42)

In [ ]:
# Simple deterministic 1D environment
class LineWorld:
    def __init__(self, size=5):
        self.size = size
        self.reset()

    def reset(self):
        self.pos = self.size // 2
        return self.pos

    def step(self, action):
        if action == 'LEFT':
            self.pos = max(0, self.pos - 1)
        elif action == 'RIGHT':
            self.pos = min(self.size - 1, self.pos + 1)
        return self.pos

def rollout(agent, internal_state, steps=5):
    env = LineWorld()
    obs = env.reset()
    traj = []
    for _ in range(steps):
        action = agent(obs, internal_state)
        obs = env.step(action)
        traj.append(action)
    return tuple(traj)

In [ ]:
# --- Agent 1: Rule‑based ---
def rule_agent(obs, state):
    return 'RIGHT' if state > 0.5 else 'LEFT'

# --- Agent 2: Tiny neural net (fixed weights) ---
W1 = np.random.randn(2, 3)
W2 = np.random.randn(3, 1)

def neural_agent(obs, state):
    x = np.array([obs / 5.0, state])
    h = np.tanh(x @ W1)
    y = h @ W2
    return 'RIGHT' if y > 0 else 'LEFT'

# --- Agent 3: Stochastic policy ---
def stochastic_agent(obs, state):
    p = 1 / (1 + np.exp(-(0.8 * state - 0.2 * obs)))
    return 'RIGHT' if np.random.rand() < p else 'LEFT'

In [ ]:
def evaluate(agent, trials=200):
    traj_A = [rollout(agent, 0.0) for _ in range(trials)]
    traj_B = [rollout(agent, 1.0) for _ in range(trials)]

    unique = list(set(traj_A + traj_B))
    counts = []
    for u in unique:
        counts.append([
            traj_A.count(u),
            traj_B.count(u)
        ])

    chi2, p, _, _ = chi2_contingency(counts)
    return len(unique), chi2, p

In [ ]:
agents = {
    'Rule‑based': rule_agent,
    'Neural': neural_agent,
    'Stochastic': stochastic_agent
}

results = {}
for name, agent in agents.items():
    results[name] = evaluate(agent)

results